# Homework 3

**Name:** Oscar Beltran Villegas

**e-mail:** oscar.beltran7944@alumnos.udg.mx

# MODULES

In [ ]:
import math
import numpy as np
from scipy.stats import wrapcauchy, levy_stable

# CLASSES

In [6]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y=None):
        if y is None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # Rotate vector
    def rotated(self, angle):
        cos_theta = math.cos(angle)
        sin_theta = math.sin(angle)
        x = self.x * cos_theta - self.y * sin_theta
        y = self.x * sin_theta + self.y * cos_theta
        return Vec2d(x, y)

# FUNCTIONS

In [10]:
def brownian_motion_3d(num_steps, speed, start_pos=(0, 0, 0)):
    """
    Function to simulate 3D Brownian motion
    Arguments:
        num_steps: Number of steps to simulate
        speed: Speed of the particle
        start_pos: Initial position of the particle
    Returns:
        trajectory: 3D trajectory of the particle
    """
    trajectory = np.zeros((num_steps + 1, 3)) # initialize the trajectory
    trajectory[0] = start_pos # set the initial position

    velocity = Vec2d(speed, 0) # initialize the velocity vector
    
    for i in range(1, num_steps + 1): # simulate the motion

        # generate a random angle to turn the velocity vector
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)
        
        # update the position
        trajectory[i] = trajectory[i-1] + np.array([velocity.x, velocity.y, velocity.get_length()])
    
    return trajectory

def correlated_random_walk_3d(num_steps, speed, start_pos=(0, 0, 0), cauchy_coefficient=0.9):
    """
    Function to simulate 3D correlated random walk
    Arguments:
        num_steps: Number of steps to simulate
        speed: Speed of the particle
        start_pos: Initial position of the particle
        cauchy_coefficient: Coefficient of the Cauchy distribution
    Returns:
        trajectory: 3D trajectory of the particle
    """
    trajectory = np.zeros((num_steps + 1, 3)) # initialize the trajectory
    trajectory[0] = start_pos # set the initial position

    velocity = Vec2d(speed, 0) # initialize the velocity vector
    
    for i in range(1, num_steps + 1): # simulate the motion

        # generate a random angle to turn the velocity vector
        angle = wrapcauchy.rvs(cauchy_coefficient)
        velocity = velocity.rotated(angle)

        # update the position
        trajectory[i] = trajectory[i-1] + np.array([velocity.x, velocity.y, velocity.get_length()])
    
    return trajectory


def levy_flight_3d(num_steps, speed, start_pos=(0, 0, 0), cauchy_coefficient=0.7, alpha=1.5):
    """
    Function to simulate 3D Levy flight
    Arguments:
        num_steps: Number of steps to simulate
        speed: Speed of the particle
        start_pos: Initial position of the particle
        cauchy_coefficient: Coefficient of the Cauchy distribution
        alpha: Alpha parameter of the Levy
    Returns:
        trajectory: 3D trajectory of the particle
    """

    beta = 0 # Beta parameter of the Levy
    time_per_step = 1.5 # Time to reach the next point

    trajectory = np.zeros((num_steps + 1, 3)) # initialize the trajectory
    trajectory[0] = start_pos # set the initial position


    velocity = Vec2d(speed,0) # initialize the velocity vector
    
    for i in range(1, num_steps + 1):
        step_length = levy_stable.rvs(alpha,beta) # Generate a random step length

        # Generate a random angle to turn the velocity vector
        angle = wrapcauchy.rvs(cauchy_coefficient)
        velocity = velocity.rotated(angle)

        # Distance / velocity = time
        time = step_length / velocity.get_length() * time_per_step # Time to reach the next point
        
        # Update the position
        trajectory[i] = trajectory[i-1] + np.array([velocity.x,velocity.y, time])
    
    return trajectory



def calculate_path_length(trajectory):
    """
    Function to calculate the path length of a trajectory
    Arguments:
        trajectory: 3D trajectory of the particle
    Returns:
        steps: Steps of the trajectory
        path_length: Path length of the trajectory
    """

    # Calculate the distance between consecutive points
    distances = np.sqrt(np.sum(np.diff(trajectory, axis=0)**2, axis=1))
    # Calculate the path length
    path_length = np.cumsum(distances)
    # Calculate the steps
    steps = np.arange(len(path_length))
    return steps, path_length

def calculate_msd(trajectory):
    """
    Function to calculate the mean squared displacement of a trajectory
    Arguments:
        trajectory: 3D trajectory of the particle
    Returns:
        steps: Steps of the trajectory
        msd: Mean squared displacement of the trajectory
    """

    steps = np.arange(len(trajectory)) # Steps of the trajectory
    start_pos = trajectory[0] # Initial position of the particle

    # Calculate the squared distances
    squared_distances = np.sum((trajectory - start_pos)**2, axis=1)
    return steps, squared_distances

def calculate_turning_angle_distribution(trajectory):
    """
    Function to calculate the turning angle distribution of
    Arguments:
        trajectory: 3D trajectory of the particle
    Returns:
        steps: Steps of the trajectory
        angles: Turning angles of the trajectory
    """
    if len(trajectory) < 3:
        return np.array([]), np.array([])
    
    # Calculate the directions of the trajectory
    directions = np.diff(trajectory, axis=0)
    
    # Calculate the angles between consecutive directions
    angles = []
    for i in range(len(directions)-1):
        v1 = directions[i]
        v2 = directions[i+1]
        
        # Normalize the vectors
        v1_norm = v1 / np.linalg.norm(v1) if np.linalg.norm(v1) > 0 else v1
        v2_norm = v2 / np.linalg.norm(v2) if np.linalg.norm(v2) > 0 else v2
        
        # Calculate the dot product
        dot_product = np.dot(v1_norm, v2_norm)
        # Clip the dot product to avoid numerical errors
        dot_product = np.clip(dot_product, -1.0, 1.0)
        
        # Calculate the angle
        angle = np.arccos(dot_product)
        angles.append(angle)
    
    steps = np.arange(len(angles))
    return steps, np.array(angles)

